In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 16:20:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 16:20:54 (8.16 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [30]:
import minsearch

In [3]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-23 16:24:06--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-06-23 16:24:06 (32.1 MB/s) - ‘documents.json’ saved [658332/658332]



In [25]:
import json

In [26]:
with open('documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [27]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [28]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [31]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [32]:
q = "the course has bugun, can I still join?"

In [33]:
index.fit(documents)

In [34]:
boost = {"question": 3.0, "section": 0.5}


results = index.search(
        query = q,
        boost_dict = boost,
        filter_dict = {"course": "data-engineering-zoomcamp"},
        num_results = 5
)

In [35]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [8]:
result = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": q}]
)

In [22]:
result.choices[0].message

ChatCompletionMessage(content='It depends on the specific course and the policies of the institution or organization offering it. Some courses allow late enrollments within a certain timeframe, while others may not accept new students once the course has started. \n\nYour best bet is to contact the course instructor or the administration office to inquire about the possibility of joining the course late. They can provide you with the most accurate information and let you know if there are any prerequisites or catch-up work that might be necessary.', role='assistant', function_call=None, tool_calls=None)

In [10]:
result.choices[0].message.content

'It depends on the specific course and the policies of the institution or organization offering it. Some courses allow late enrollments within a certain timeframe, while others may not accept new students once the course has started. \n\nYour best bet is to contact the course instructor or the administration office to inquire about the possibility of joining the course late. They can provide you with the most accurate information and let you know if there are any prerequisites or catch-up work that might be necessary.'

In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FSQ database.
Use only the facts from the CONTEXT when asnwering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}
"""
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()


In [40]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FSQ database.
Use only the facts from the CONTEXT when asnwering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has bugun, can I still join?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register

In [41]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can still join the course even if it has already begun. You are eligible to submit the homework but be aware of the deadlines for turning in the final projects. Make sure not to leave everything for the last minute.'